In [1]:
from mne.io import concatenate_raws, read_raw_edf
import mne
import glob
import numpy as np
import xlrd
import pandas as pd

In [2]:
path = "../SE_parameter/stimulate_parameter.xlsx"
df = pd.read_excel(path,engine='openpyxl')
#df= df.astype({'完成时间':'object'})
#对df进行排序，为了和glob.glob的顺序相匹配
df=df.sort_values(by=["姓名","完成时间"])

In [3]:
df=df.reset_index()
df=df.drop(['index','Unnamed: 0'],axis=1)

In [4]:
df.shape

(192, 5)

In [5]:
ECT_code = []

In [6]:
for i in range(df.shape[0]):
    ECT_code.append(df.iloc[i,3])

In [7]:
ECT_code = np.array(ECT_code)

In [8]:
ECT_code

array([1, 4, 3, 2, 4, 1, 2, 3, 3, 1, 4, 2, 3, 4, 2, 1, 4, 2, 1, 3, 4, 2,
       3, 1, 2, 4, 1, 3, 1, 2, 3, 4, 2, 1, 4, 3, 2, 4, 1, 3, 2, 1, 3, 4,
       4, 3, 1, 2, 3, 2, 1, 4, 2, 3, 1, 4, 3, 1, 2, 4, 2, 3, 4, 1, 3, 4,
       1, 2, 2, 1, 4, 3, 3, 1, 4, 2, 4, 1, 2, 3, 1, 3, 2, 4, 4, 3, 1, 2,
       2, 1, 3, 4, 1, 3, 2, 4, 4, 1, 2, 3, 1, 4, 2, 3, 4, 3, 2, 1, 3, 1,
       2, 4, 2, 4, 3, 1, 4, 2, 3, 1, 3, 4, 1, 2, 2, 3, 4, 1, 3, 2, 1, 4,
       1, 2, 3, 4, 3, 2, 4, 1, 4, 2, 3, 1, 2, 1, 4, 3, 2, 4, 1, 3, 1, 2,
       3, 4, 2, 4, 3, 1, 1, 3, 4, 2, 3, 4, 1, 2, 4, 1, 3, 2, 2, 3, 1, 4,
       1, 2, 4, 3, 1, 3, 4, 2, 1, 4, 2, 3, 3, 4, 2, 1], dtype=int64)

In [9]:
# rest_path = r"../patient_data/静息脑电（第一批次）/"
therapy_path=r"../patient_data/治疗脑电/"

# rest_subdirectory = sorted(glob.glob(rest_path+"*"))
# therapy_subdirectory = sorted(glob.glob(therapy_path+"*"))
therapy_file_path = sorted(glob.glob(therapy_path+"*"))

In [10]:
len(therapy_file_path)

190

In [11]:
path_parameter_dict={}

In [12]:
len(therapy_file_path)

190

In [13]:
#构建一个 {路径:ECT参数} 的字典，方便打乱顺序后，仍能保持一一对应
for i in range(len(therapy_file_path)):
    path_parameter_dict[therapy_file_path[i]]=ECT_code[i]

In [14]:
path_parameter_dict

{'../patient_data/治疗脑电\\Z何富桂20220221.edf': 1,
 '../patient_data/治疗脑电\\Z何富桂20220225.edf': 4,
 '../patient_data/治疗脑电\\Z何富桂20220228.edf': 3,
 '../patient_data/治疗脑电\\Z何富桂20220302.edf': 2,
 '../patient_data/治疗脑电\\Z何金蓓20220511.edf': 4,
 '../patient_data/治疗脑电\\Z何金蓓20220516（未发作）.edf': 1,
 '../patient_data/治疗脑电\\Z何金蓓20220518（未发作）.edf': 2,
 '../patient_data/治疗脑电\\Z何金蓓20220520（未发作）.edf': 3,
 '../patient_data/治疗脑电\\Z余琛宇20220411.edf': 3,
 '../patient_data/治疗脑电\\Z余琛宇20220413.edf': 1,
 '../patient_data/治疗脑电\\Z余琛宇20220415.edf': 4,
 '../patient_data/治疗脑电\\Z余琛宇20220422.edf': 2,
 '../patient_data/治疗脑电\\Z倪诗瑄20220311.edf': 3,
 '../patient_data/治疗脑电\\Z倪诗瑄20220314.edf': 4,
 '../patient_data/治疗脑电\\Z倪诗瑄20220316.edf': 2,
 '../patient_data/治疗脑电\\Z倪诗瑄20220318.edf': 1,
 '../patient_data/治疗脑电\\Z刘世清20220309.edf': 4,
 '../patient_data/治疗脑电\\Z刘世清20220311.edf': 2,
 '../patient_data/治疗脑电\\Z刘世清20220314（未发作）.edf': 1,
 '../patient_data/治疗脑电\\Z刘世清20220316.edf': 3,
 '../patient_data/治疗脑电\\Z刘家瑞20220225.edf': 4,
 '../patient_d

In [15]:
#观察文件名，发现所有文件名均为“Z人名日期”，所以取文件名的[1:4]为人名，前后对照，如果不同则表示是不同的人
#思路：首先将所有人名取出，然后随机打乱顺序，前20个人作为训练数据，后4个人作为测试数据
therapy_file_path[0].replace('\\','/').split('/')[-1][1:4]

'何富桂'

In [16]:
#获得名单
def getNameList(file_path,name_list):
    last_name='1'
    for i in file_path:
        cur_name=i.replace('\\','/').split('/')[-1][1:4]
        if (last_name!=cur_name):
            name_list.append(cur_name)
        last_name=cur_name      

In [27]:
#打乱名单，体现随机性
np.random.seed(42)
name_list=[]
getNameList(therapy_file_path,name_list)
np.random.shuffle(name_list)
name_list

['王文珊',
 '陈莎莎',
 '王宜凡',
 '黄晓旸',
 '沈伟2',
 '邵子健',
 '宋志辉',
 '杜娇2',
 '刘世清',
 '沈祺雨',
 '周博2',
 '倪诗瑄',
 '刘浩天',
 '陈琳2',
 '肖星星',
 '张璐2',
 '李振千',
 '黄金燕',
 '曾刚2',
 '周小凯',
 '李坤铣',
 '王胜银',
 '程婉君',
 '齐金钢',
 '何富桂',
 '祁燕2',
 '王莹2',
 '刘家瑞',
 '孙彭辉',
 '肖智华',
 '何金蓓',
 '黄盈雪',
 '林子幸',
 '余琛宇',
 '赵丽兰',
 '裴瑞婷',
 '柯志祥',
 '雷振2',
 '周牧涵',
 '林攀攀',
 '李玉萍',
 '龙园芳',
 '林儒波',
 '南添仁',
 '魏平2',
 '徐珍英',
 '王纯刚',
 '陈倩2']

In [28]:
len(name_list)

48

In [29]:
#此处设定有多少患者用于训练集(人数)
train_patient_num=40
total_patient_num=len(name_list)
train_therapy_file_path=[]
test_therapy_file_path=[]
for i in range(train_patient_num):
    temp_file_path=sorted(glob.glob(therapy_path+'*'+name_list[i]+'*'))
    for j in temp_file_path:
        train_therapy_file_path.append(j)
for i in range(train_patient_num,total_patient_num):
    temp_file_path=sorted(glob.glob(therapy_path+'*'+name_list[i]+'*'))
    for j in temp_file_path:
        test_therapy_file_path.append(j)

In [30]:
#这里必须要排序
train_therapy_file_path.sort()
test_therapy_file_path.sort()

In [31]:
#统计未发作和发作的样本数
trigger_num=0
non_trigger_num=0
for i in train_therapy_file_path:
    # 找不到未发作，即发作
    if i.find('未发作')==-1:
        trigger_num+=1
    else:
        non_trigger_num+=1
print(trigger_num/non_trigger_num)
#例如发作样本采集SE后8s，multiple=5，那么未发作样本就从offset=2,4,6,8,10采集8s SE片段
multiple=trigger_num/non_trigger_num

3.1842105263157894


In [35]:
multiple = round(multiple)
multiple

3

In [36]:
non_trigger_num

38

In [37]:
trigger_num+non_trigger_num*(int)(multiple)

235

In [38]:
#检测EEG文件是否有flag标记,返回true/false
def judgeflag(file_path,flag):
    raw = read_raw_edf(file_path,preload=False)
    events_from_annot, event_dict = mne.events_from_annotations(raw)
    if flag in event_dict:
        return True
    else:
        return False

In [39]:
# 数据采样率1024Hz,文件路径；采集时间；麻醉起始点的偏移(s);是否为测试集标志
def get_se_epochs(path,select_time,bias,is_test):
    #选择15个通道
    chs1 = ['Fp1-AV', 'Fp2-AV', 'F3-AV', 'F4-AV', 'C3-AV', 'C4-AV', 'P3-AV', 'P4-AV',
       'O1-AV', 'O2-AV', 'T3-AV', 'T4-AV', 'T5-AV', 'T6-AV','Cz-AV']
    chs2 =['Fp1-A1', 'Fp2-A2', 'F3-A1','F4-A2','C3-A1','C4-A2','P3-A1','P4-A2','O1-A1',
           'O2-A2','T3-A1','T4-A2','T5-A1','T6-A2','Cz-AAV']
    # select_time 采集的时间
    raw=read_raw_edf(path,preload=False)
    # print(raw.info.ch_names)
    if chs1[0] in raw.info.ch_names:
        chs=chs1  
    else:
        chs=chs2
    # print(chs)
    raw=raw.pick_channels(chs)
    events_from_annot, event_dict = mne.events_from_annotations(raw)
    # 第二个标记是SE，麻醉标记,save函数的参数单位是s，所以要除以1024
    start_time=float(events_from_annot[2][0])/1024 + bias
    end_time=start_time+select_time
    print(start_time,"+",end_time)
    #选择所有的15通道
    data,time = raw[:,:] 
    picks=None
    path_name=path.split('\\')[1:]
    path_name="".join(path_name)
    path_name=path_name.split('.')[0]
    #在这里将训练集和测试集分开,需要新建SE_train和SE_test两个文件夹
    if (is_test):      
        path_name=r"../SE_test/" + path_name + ".fif"
    else:
        #此处不可写成_bias: 因为命名规则里面不能写  ':'
        path_name=r"../SE_train/" + path_name + "_bias_" + str(bias) +".fif"
    path_name=path_name.replace('/','\\')
    print(path_name)
    raw.save(path_name,picks,start_time,end_time,overwrite=True)

In [40]:
file_without_se = []
df_parameter_train=pd.DataFrame(columns=['parameter'])
df_parameter_test=pd.DataFrame(columns=['parameter'])

In [41]:
#生成SE_train
for file_path in train_therapy_file_path:
    if judgeflag(file_path,"SE"):
        #如果发作，则只截取8s 
        #为False时表示当前数据送入训练集
        get_se_epochs(file_path,8,0,False)
        #添加参数到df_parameter_train的末尾
        df_parameter_train.loc[len(df_parameter_train.index)]=path_parameter_dict[file_path]
        #如果发作，则再截取offset为8,16,24……的8s片段
        if file_path.find('未发作')!=-1:
            for i in range(8,8*(int)(multiple),8):
                get_se_epochs(file_path,8,i,False) 
                df_parameter_train.loc[len(df_parameter_train.index)]=path_parameter_dict[file_path]
    else:
        file_without_se.append(file_path)
#生成SE_test,不考虑对未发作数据的重叠采样
for file_path in test_therapy_file_path:
    if judgeflag(file_path,"SE"):
        #仍然取16s片段
        get_se_epochs(file_path,16,0,True) 
        df_parameter_test.loc[len(df_parameter_test.index)]=path_parameter_dict[file_path]
    else:
        file_without_se.append(file_path)

Extracting EDF parameters from H:\Anaconda-jupyter\ECT课题\EEGprocess-main\patient_data\治疗脑电\Z何富桂20220221.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['File Create', 'Recording ends', 'Recording starts', 'SE', 'end', 'start']
Extracting EDF parameters from H:\Anaconda-jupyter\ECT课题\EEGprocess-main\patient_data\治疗脑电\Z何富桂20220221.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['File Create', 'Recording ends', 'Recording starts', 'SE', 'end', 'start']
8.7099609375 + 16.7099609375
..\SE_train\Z何富桂20220221_bias_0.fif
Writing H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..\SE_train\Z何富桂20220221_bias_0.fif
Closing H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..\SE_train\Z何富桂20220221_bias_0.fif
[done]
Extracting EDF parameters from H:\Anaconda-jupyter\ECT课题\EEGprocess-main\patient_data\治疗脑电\Z何富桂20220225.edf...
EDF file detected
Setti

<ipython-input-39-26fe23d1e1e9>:36: RuntimeWarning: This filename (H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..\SE_train\Z何富桂20220221_bias_0.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(path_name,picks,start_time,end_time,overwrite=True)
<ipython-input-39-26fe23d1e1e9>:36: RuntimeWarning: This filename (H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..\SE_train\Z何富桂20220225_bias_0.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(path_name,picks,start_time,end_time,overwrite=True)


Closing H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..\SE_train\Z何富桂20220225_bias_0.fif
[done]
Extracting EDF parameters from H:\Anaconda-jupyter\ECT课题\EEGprocess-main\patient_data\治疗脑电\Z何富桂20220228.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['File Create', 'Recording ends', 'Recording starts', 'SE', 'end', 'start']
Extracting EDF parameters from H:\Anaconda-jupyter\ECT课题\EEGprocess-main\patient_data\治疗脑电\Z何富桂20220228.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['File Create', 'Recording ends', 'Recording starts', 'SE', 'end', 'start']
10.33203125 + 18.33203125
..\SE_train\Z何富桂20220228_bias_0.fif
Writing H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..\SE_train\Z何富桂20220228_bias_0.fif
Closing H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..\SE_train\Z何富桂20220228_bias_0.fif
[done]
Extracting EDF parameters from H:\

<ipython-input-39-26fe23d1e1e9>:36: RuntimeWarning: This filename (H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..\SE_train\Z何富桂20220228_bias_0.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(path_name,picks,start_time,end_time,overwrite=True)
<ipython-input-39-26fe23d1e1e9>:36: RuntimeWarning: This filename (H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..\SE_train\Z何富桂20220302_bias_0.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(path_name,picks,start_time,end_time,overwrite=True)


142.951171875 + 150.951171875
..\SE_train\Z何金蓓20220511_bias_0.fif
Writing H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..\SE_train\Z何金蓓20220511_bias_0.fif
Closing H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..\SE_train\Z何金蓓20220511_bias_0.fif
[done]
Extracting EDF parameters from H:\Anaconda-jupyter\ECT课题\EEGprocess-main\patient_data\治疗脑电\Z何金蓓20220516（未发作）.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['File Create', 'Recording ends', 'Recording starts', 'SE']
Extracting EDF parameters from H:\Anaconda-jupyter\ECT课题\EEGprocess-main\patient_data\治疗脑电\Z何金蓓20220516（未发作）.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['File Create', 'Recording ends', 'Recording starts', 'SE']
17.5966796875 + 25.5966796875


...

<ipython-input-39-26fe23d1e1e9>:36: RuntimeWarning: This filename (H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..\SE_test\Z魏平20220427.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(path_name,picks,start_time,end_time,overwrite=True)
<ipython-input-39-26fe23d1e1e9>:36: RuntimeWarning: This filename (H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..\SE_test\Z魏平20220429.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(path_name,picks,start_time,end_time,overwrite=True)


179.0 + 195.0
..\SE_test\Z魏平20220509（未发作）.fif
Writing H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..\SE_test\Z魏平20220509（未发作）.fif
Closing H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..\SE_test\Z魏平20220509（未发作）.fif
[done]
Extracting EDF parameters from H:\Anaconda-jupyter\ECT课题\EEGprocess-main\patient_data\治疗脑电\Z魏平20220513.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE', 'end', 'start']
Extracting EDF parameters from H:\Anaconda-jupyter\ECT课题\EEGprocess-main\patient_data\治疗脑电\Z魏平20220513.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE', 'end', 'start']
67.9140625 + 83.9140625
..\SE_test\Z魏平20220513.fif
Writing H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..\SE_test\Z魏平20220513.fif
Closing H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_

<ipython-input-39-26fe23d1e1e9>:36: RuntimeWarning: This filename (H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..\SE_test\Z魏平20220509（未发作）.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(path_name,picks,start_time,end_time,overwrite=True)
<ipython-input-39-26fe23d1e1e9>:36: RuntimeWarning: This filename (H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..\SE_test\Z魏平20220513.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(path_name,picks,start_time,end_time,overwrite=True)
<ipython-input-39-26fe23d1e1e9>:36: RuntimeWarning: This filename (H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..

187.0 + 203.0
..\SE_test\Z龙园芳20220511（未发作）.fif
Writing H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..\SE_test\Z龙园芳20220511（未发作）.fif
Closing H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..\SE_test\Z龙园芳20220511（未发作）.fif
[done]
Extracting EDF parameters from H:\Anaconda-jupyter\ECT课题\EEGprocess-main\patient_data\治疗脑电\Z龙园芳20220513（未发作）.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE']
Extracting EDF parameters from H:\Anaconda-jupyter\ECT课题\EEGprocess-main\patient_data\治疗脑电\Z龙园芳20220513（未发作）.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['Recording ends', 'Recording starts', 'SE']
111.0 + 127.0
..\SE_test\Z龙园芳20220513（未发作）.fif
Writing H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..\SE_test\Z龙园芳20220513（未发作）.fif
Closing H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..\SE_test

<ipython-input-39-26fe23d1e1e9>:36: RuntimeWarning: This filename (H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..\SE_test\Z龙园芳20220511（未发作）.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(path_name,picks,start_time,end_time,overwrite=True)
<ipython-input-39-26fe23d1e1e9>:36: RuntimeWarning: This filename (H:\Anaconda-jupyter\ECT课题\EEGprocess-main\work_file\..\SE_test\Z龙园芳20220513（未发作）.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw.save(path_name,picks,start_time,end_time,overwrite=True)


In [42]:
file_without_se

['../patient_data/治疗脑电\\Z刘家瑞20220304.edf',
 '../patient_data/治疗脑电\\Z周博20220420.edf',
 '../patient_data/治疗脑电\\Z张璐20220304.edf',
 '../patient_data/治疗脑电\\Z李坤铣20220523.edf',
 '../patient_data/治疗脑电\\Z李振千20220617（未发作）.edf',
 '../patient_data/治疗脑电\\Z林攀攀20220221（未发作）.edf',
 '../patient_data/治疗脑电\\Z沈祺雨20220516（未发作）.edf',
 '../patient_data/治疗脑电\\Z王胜银20220523（未发作）.edf',
 '../patient_data/治疗脑电\\Z肖智华20220307（未发作）.edf',
 '../patient_data/治疗脑电\\Z邵子健20220509（未发作）.edf',
 '../patient_data/治疗脑电\\Z陈莎莎20220415.edf',
 '../patient_data/治疗脑电\\Z黄晓旸20220509.edf']

In [43]:
df_parameter_train

,parameter
0,1
1,4
2,3
3,2
4,4
...,...
206,4
207,2
208,1
209,4


In [44]:
df_parameter_test

,parameter
0,1
1,2
2,3
3,4
4,3
5,1
6,2
7,4
8,3
9,1


In [45]:
train_output_path=("../SE_parameter/train_matched_stimulate_parameter.xlsx")
test_output_path=("../SE_parameter/test_matched_stimulate_parameter.xlsx")

In [46]:
df_parameter_train.to_excel(train_output_path,sheet_name="Sheet1")

In [47]:
df_parameter_test.to_excel(test_output_path,sheet_name="Sheet1")